In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
from tensorflow import keras
import os
from sklearn import model_selection
from sklearn import utils
from sklearn.metrics import confusion_matrix, roc_curve, RocCurveDisplay, roc_auc_score
from scipy import ndimage
from gc import collect
import matplotlib.pyplot as plt
from collections import Counter
from seaborn import heatmap

In [ ]:
def get_labels(label_file):
    """lê a tabela com as informações dos pacientes e retorna uma matriz com o ID e as labels"""
    labels = pd.read_csv(label_file)
    cancer_labels = dict()

    for p in labels.index:
        cancer_labels[labels['PatientID'][p]] = [int(labels['Normal'][p]), int(labels['Actionable'][p]), int(labels['Benign'][p]), int(labels['Cancer'][p])]

    return cancer_labels

In [ ]:
labels = get_labels('/kaggle/input/labelsssssss/labels.csv')
numbers_per_class = [0, 0, 0, 0]
for i in labels:
    numbers_per_class[np.argmax(labels[i])] += 1

proportion_per_class = [round(number_of_class/sum(numbers_per_class), 2) for number_of_class in numbers_per_class]
proportion_per_class

In [ ]:
path = '/kaggle/input/192x256xdepth/'
dirs = os.listdir(path)

X = [path + i for i in os.listdir(path)]
y = [np.argmax(labels[(path+i)[-14:-4]]) for i in os.listdir(path)]

In [ ]:
def balancing_batch(X, y, max_value):
    numbers_per_class = sorted(Counter(y).items())

    if len(numbers_per_class) == 1:
        return X[0:1], y[0:1] #return only the first image because the batch has only one class

    if numbers_per_class[0][1] > numbers_per_class[1][1]:
        max_per_class = numbers_per_class[1][1]
    else:
        max_per_class = numbers_per_class[0][1]

    if max_per_class > max_value//2:
        max_per_class = max_value//2

    X, y = utils.shuffle(X, y)
    new_X, new_y = [], []
    counter_class_zero = 0
    counter_class_one = 0
    for test_x, test_y in zip(X, y):
        if test_y == 0 and counter_class_zero < max_per_class//2:
            new_X.append(test_x)
            new_y.append(test_y)
            counter_class_zero += 1
        elif test_y == 1 and counter_class_one < max_per_class//2:
            new_X.append(test_x)
            new_y.append(test_y)
            counter_class_one += 1

    return np.array(new_X, dtype='float16'), np.array(new_y, dtype='float16')

In [ ]:
def separate_slices(img):
    'function to separate 2d images of 3d original image'
    slices = []

    for i in range(img.shape[-2]):
        slices.append(np.array(img[:, :, i]))

    slices.append(np.mean(img, axis=-2)) #including mean of slices

    return slices

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, stratify=y,random_state=42, train_size=0.8)

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, objective, list_IDs, labels_dir, batch_size, sub_batch_size, dim_img, training, shuffle=True):
        self.objective = objective # list representing one hot encoding for choice label ([1, 0, 0, 0] for normal, [0, 0, 0, 1] for cancer ..)
        self.list_IDs = list_IDs # array of strings with original images name with directory
        self.labels = self.__get_labels(labels_dir) #dict with labels of all images
        self.batch_size = batch_size #3d-images per batch
        self.sub_batch_size = sub_batch_size #quantity of sub-images per batch will be choose to train
        self.dim_img = dim_img # tuple with width and height of image like (192, 256)
        self.training = training # true if generator is for training, false if generator is for validation
        self.shuffle = shuffle # true or false to shuffle data after any epochs
        self.on_epoch_end() # call of the function


    def __get_labels(self, label_file):
        'take the dict with labels of images'
        labels = pd.read_csv(label_file)
        cancer_labels = dict()

        for p in labels.index:
            cancer_labels[labels['PatientID'][p]] = [int(labels['Normal'][p]), int(labels['Actionable'][p]), int(labels['Benign'][p]), int(labels['Cancer'][p])]

        return cancer_labels

    def __data_augmentation(self, x):
        'generate variations of images'
        new_images = []
        x = x.astype('float16')
        new_images.append(x)

        x = cv2.flip(x.astype('float32'), 1).astype('float16')

        new_images.append(np.expand_dims(x, -1))

        return utils.shuffle(new_images)

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))


    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        if self.training:
            X, y = balancing_batch(X, y, self.sub_batch_size)
            return np.array(X[0:self.sub_batch_size], dtype='float16'), np.array(y[0:self.sub_batch_size], dtype='uint8')

        return np.array(X), np.array(y)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'
        X = []
        y = []

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            prev_len_X = len(X)
            full_image = np.load(ID)
            new_images = separate_slices(full_image)
            if self.training:
                for img in new_images:
                    X += self.__data_augmentation(img)
            else:
                X = np.array(new_images, dtype='float16')

            #adding new data labels for y array
            for _ in range(len(X) - prev_len_X):
                if self.labels[ID[-14:-4]] == self.objective: #'-14:-4 represent a part of string with name of original image that slices was taken'
                    y.append(1)
                else:
                    y.append(0)

        X, y = utils.shuffle(X, y)
        return X, y

In [ ]:
class ConvLayer(keras.layers.Layer):
    def __init__(self, filters, kernel_x, kernel_y, activation='relu', strides=1):
        super().__init__()

        self.conv1 = keras.layers.Conv2D(filters=filters, kernel_size=(1,1), padding='same')
        self.conv2 = keras.layers.Conv2D(filters=filters, kernel_size=(kernel_x, kernel_y), padding='same', strides=strides,)
        self.bn    = keras.layers.BatchNormalization()
        self.activation = keras.layers.get_activation(activation)

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.bn(x)
        x = self.activation(x)
        return x

class DenseBlock(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.conv_layers = [ConvLayer(filters, 2, 4) for _ in range(16)]

    actual_input = x
    def call(self, x):
      for conv_layer in self.conv_layers:
        output = conv_layer(atual_input)
        actual_input = tf.concat([actual_input, output], axis=-1)
      return output

class DenseNET(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv   = keras.layers.Conv2D(filters=16, kernel_size=(7, 9), strides=2, activation='relu', padding='same')
        self.bn1    = keras.layers.BatchNormalization()
        self.block1 = DenseBlock(32)
        self.conv1  = ConvLayer(64, 1, 1)
        self.pool1  = keras.layers.MaxPooling2D(kernel=(2,2), strides=2)
        self.block2 = DenseBlock(64)
        self.conv2  = ConvLayer(64, 1, 1)
        self.pool2  = keras.layers.MaxPooling2D(kernel=(2,2), strides=2)
        self.block3 = DenseBlock(128)
        self.pool3  = keras.layers.GlobalAveragePooling2D()
        self.dense = keras.layers.Dense(units=1, activation='sigmoid')

    def call(self, x):
        x = self.conv(x)
        x = self.bn1(x)
        x = self.block1(x)
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.block2(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.block3(x)
        x = self.pool3(x)
        x = self.dense(x)

      return x

NORMAL

In [ ]:
checkpoint_callback_auc1 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/normal_auc1/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_1',
                                                             mode='max',
                                                             save_best_only=True)
checkpoint_callback_auc2 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/normal_auc2/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_2',
                                                             mode='max',
                                                             save_best_only=True)
checkpoint_callback_auc3 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/normal_auc3/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_3',
                                                             mode='max',
                                                             save_best_only=True)
checkpoint_callback_auc4 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/normal_auc4/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_4',
                                                             mode='max',
                                                             save_best_only=True)
checkpoint_callback_auc5 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/normal_auc5/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_5',
                                                             mode='max',
                                                             save_best_only=True)

callbacks_list = [checkpoint_callback_auc1,
                  checkpoint_callback_auc2,
                  checkpoint_callback_auc3,
                  checkpoint_callback_auc4,
                  checkpoint_callback_auc5]

def lr_scheduler(epoch, lr):
    return lr*0.9

lr_decay_function = keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=0)

In [ ]:
histories = []
n_splits = 5
skf = model_selection.StratifiedShuffleSplit(n_splits=n_splits, random_state=314, train_size=0.85)
for number_of_split, data in enumerate(skf.split(X_train, y_train)):
    print(f'SPLIT {number_of_split+1}/{n_splits}:')
    train = [path + dirs[i] for i in data[0]]
    val = [path + dirs[j] for j in data[1]]

    # Generators
    training_generator = DataGenerator(objective=[1, 0, 0, 0],
                                       list_IDs=train,
                                       labels_dir='/kaggle/input/labelsssssss/labels.csv',
                                       dim_img=(192, 256),
                                       batch_size=5,
                                       sub_batch_size=400,
                                       shuffle=True,
                                       training=True)

    validation_generator = DataGenerator(objective=[1, 0, 0, 0],
                                         list_IDs=val,
                                         labels_dir='/kaggle/input/labelsssssss/labels.csv',
                                         dim_img=(192, 256),
                                         batch_size=1,
                                         sub_batch_size='IGNORED', #this argument will be ignored because training is false.
                                         shuffle=True,
                                         training=False)

    densenet = DenseNET()

    densenet.compile(loss='binary_crossentropy',
                     optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                     metrics=[keras.metrics.AUC(name=f'AUC_{number_of_split+1}'),
                              keras.metrics.BinaryAccuracy(name='accuracy'),
                              keras.metrics.Precision(name='precision'),
                              keras.metrics.Recall(name='recall')])


    # Train model on dataset
    histories.append(densenet.fit(training_generator,
                                   validation_data=validation_generator,
                                   epochs=35,
                                   use_multiprocessing=True,
                                   workers=1,
                                   callbacks=[callbacks_list[number_of_split], lr_decay_function]))
    print('\n')
    collect()

In [ ]:
plt.figure(figsize=(25, 50))
plt.subplot(4, 2, 1)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['loss'], marker='o')
    plt.title('DenseNET Loss evolution - Training: Normal vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Loss on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 2)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['val_loss'], marker='o')
    plt.title('DenseNET4 Loss evolution - Validations: Normal vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Loss on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 3)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['accuracy'], marker='o')
    plt.title('DenseNET Accuracy evolution - Training: Normal vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Accuracy on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 4)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['val_accuracy'], marker='o')
    plt.title('DenseNET Accuracy evolution - Validations: Normal vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Accuracy on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 5)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['recall'], marker='o')
    plt.title('DenseNET Recall evolution - Training: Normal vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Recall on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 6)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['val_recall'], marker='o')
    plt.title('DenseNET Recall evolution - Validations: Normal vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Recall on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 7)
for i, h in enumerate(histories):
    key_auc = f"AUC_{i+1}"
    plt.plot(list(range(1, 36)), h.history[key_auc], marker='o')
    plt.title('DenseNET AUC evolution - Training: Normal vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('AUC on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 8)
for i, h in enumerate(histories):
    key_auc_val = f"val_AUC_{i+1}"
    plt.plot(list(range(1, 36)), h.history[key_auc_val], marker='o')
    plt.title('DenseNET AUC evolution - Validations: Normal vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('AUC on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

In [ ]:
print('Best results in validations for any k-fold: ')
for i, h in enumerate(histories):
    print(f'K-FOLD {i+1}:')
    print("TRAINING RESULTS:")
    k = np.max(h.history[f'AUC_{i+1}'])
    print(f'Best AUC in train: {k}')
    k = np.max(h.history[f'accuracy'])
    print(f'Best Accuracy in train: {k}')
    k = np.max(h.history[f'precision'])
    print(f'Best Precision in train: {k}')
    k = np.max(h.history[f'recall'])
    print(f'Best Recall in train: {k}')

    print("\nVALIDATION RESULTS:")
    k = np.max(h.history[f'val_AUC_{i+1}'])
    print(f'Best AUC in validation: {k}')
    k = np.max(h.history[f'val_accuracy'])
    print(f'Best Accuracy in validation: {k}')
    k = np.max(h.history[f'val_precision'])
    print(f'Best Precision in validation: {k}')
    k = np.max(h.history[f'val_recall'])
    print(f'Best Recall in validation: {k}')
    print()
    print(f'{50*"="}')
    print()

results = np.empty((4, 5))
for i, h in enumerate(histories):
    results[0][i] = np.max(h.history[f'val_AUC_{i+1}'])
    results[1][i] = np.max(h.history['val_accuracy'])
    results[2][i] = np.max(h.history['val_precision'])
    results[3][i] = np.max(h.history['val_recall'])

print(f"Average best AUC: {np.mean(results[0])}")
print(f"standard deviation AUC: {np.std(results[0])}\n")
print(f"Average best Accuracy: {np.mean(results[1])}")
print(f"Standard Deviation Accuracy: {np.std(results[1])}\n")
print(f"Average best Precision: {np.mean(results[2])}")
print(f"Standard Deviation Precision: {np.std(results[2])}\n")
print(f"Average best Recall: {np.mean(results[3])}")
print(f"Standard Deviation Recall: {np.std(results[3])}\n")


ACTIONABLE

In [ ]:
checkpoint_callback_auc1 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/actionable_auc1/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_1',
                                                             mode='max',
                                                             save_best_only=True)
checkpoint_callback_auc2 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/actionable_auc2/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_2',
                                                             mode='max',
                                                             save_best_only=True)
checkpoint_callback_auc3 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/actionable_auc3/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_3',
                                                             mode='max',
                                                             save_best_only=True)
checkpoint_callback_auc4 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/actionable_auc4/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_4',
                                                             mode='max',
                                                             save_best_only=True)
checkpoint_callback_auc5 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/actionable_auc5/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_5',
                                                             mode='max',
                                                             save_best_only=True)

callbacks_list = [checkpoint_callback_auc1,
                  checkpoint_callback_auc2,
                  checkpoint_callback_auc3,
                  checkpoint_callback_auc4,
                  checkpoint_callback_auc5]

def lr_scheduler(epoch, lr):
    return lr*0.9

lr_decay_function = keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=0)

In [ ]:
histories = []
n_splits = 5
skf = model_selection.StratifiedShuffleSplit(n_splits=n_splits, random_state=314, train_size=0.85)
for number_of_split, data in enumerate(skf.split(X_train, y_train)):
    print(f'SPLIT {number_of_split+1}/{n_splits}:')
    train = [path + dirs[i] for i in data[0]]
    val = [path + dirs[j] for j in data[1]]

    # Generators
    training_generator = DataGenerator(objective=[0, 1, 0, 0],
                                       list_IDs=train,
                                       labels_dir='/kaggle/input/labelsssssss/labels.csv',
                                       dim_img=(192, 256),
                                       batch_size=5,
                                       sub_batch_size=400,
                                       shuffle=True,
                                       training=True)

    validation_generator = DataGenerator(objective=[0, 1, 0, 0],
                                         list_IDs=val,
                                         labels_dir='/kaggle/input/labelsssssss/labels.csv',
                                         dim_img=(192, 256),
                                         batch_size=1,
                                         sub_batch_size='IGNORED', #this argument will be ignored because training is false.
                                         shuffle=True,
                                         training=False)

    densenet = DenseNET()

    densenet.compile(loss='binary_crossentropy',
                     optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                     metrics=[keras.metrics.AUC(name=f'AUC_{number_of_split+1}'),
                              keras.metrics.BinaryAccuracy(name='accuracy'),
                              keras.metrics.Precision(name='precision'),
                              keras.metrics.Recall(name='recall')])


    # Train model on dataset
    histories.append(densenet.fit(training_generator,
                                   validation_data=validation_generator,
                                   epochs=35,
                                   use_multiprocessing=True,
                                   workers=1,
                                   callbacks=[callbacks_list[number_of_split], lr_decay_function]))
    print('\n')
    collect()

In [ ]:
plt.figure(figsize=(25, 50))
plt.subplot(4, 2, 1)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['loss'], marker='o')
    plt.title('DenseNET Loss evolution - Training: Actionable vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Loss on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 2)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['val_loss'], marker='o')
    plt.title('DenseNET Loss evolution - Validations: Actionable vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Loss on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 3)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['accuracy'], marker='o')
    plt.title('DenseNET Accuracy evolution - Training: Actionable vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Accuracy on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 4)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['val_accuracy'], marker='o')
    plt.title('DenseNET Accuracy evolution - Validations: Actionable vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Accuracy on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 5)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['recall'], marker='o')
    plt.title('DenseNET Recall evolution - Training: Actionable vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Recall on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 6)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['val_recall'], marker='o')
    plt.title('DenseNET Recall evolution - Validations: Actionable vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Recall on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 7)
for i, h in enumerate(histories):
    key_auc = f"AUC_{i+1}"
    plt.plot(list(range(1, 36)), h.history[key_auc], marker='o')
    plt.title('DenseNET AUC evolution - Training: Actionable vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('AUC on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 8)
for i, h in enumerate(histories):
    key_auc_val = f"val_AUC_{i+1}"
    plt.plot(list(range(1, 36)), h.history[key_auc_val], marker='o')
    plt.title('DenseNET AUC evolution - Validations: Actionable vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('AUC on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

In [ ]:
print('Best results in validations for any k-fold: ')
for i, h in enumerate(histories):
    print(f'K-FOLD {i+1}:')
    print("TRAINING RESULTS:")
    k = np.max(h.history[f'AUC_{i+1}'])
    print(f'Best AUC in train: {k}')
    k = np.max(h.history[f'accuracy'])
    print(f'Best Accuracy in train: {k}')
    k = np.max(h.history[f'precision'])
    print(f'Best Precision in train: {k}')
    k = np.max(h.history[f'recall'])
    print(f'Best Recall in train: {k}')

    print("\nVALIDATION RESULTS:")
    k = np.max(h.history[f'val_AUC_{i+1}'])
    print(f'Best AUC in validation: {k}')
    k = np.max(h.history[f'val_accuracy'])
    print(f'Best Accuracy in validation: {k}')
    k = np.max(h.history[f'val_precision'])
    print(f'Best Precision in validation: {k}')
    k = np.max(h.history[f'val_recall'])
    print(f'Best Recall in validation: {k}')
    print()
    print(f'{50*"="}')
    print()

results = np.empty((4, 5))
for i, h in enumerate(histories):
    results[0][i] = np.max(h.history[f'val_AUC_{i+1}'])
    results[1][i] = np.max(h.history['val_accuracy'])
    results[2][i] = np.max(h.history['val_precision'])
    results[3][i] = np.max(h.history['val_recall'])

print(f"Average best AUC: {np.mean(results[0])}")
print(f"standard deviation AUC: {np.std(results[0])}\n")
print(f"Average best Accuracy: {np.mean(results[1])}")
print(f"Standard Deviation Accuracy: {np.std(results[1])}\n")
print(f"Average best Precision: {np.mean(results[2])}")
print(f"Standard Deviation Precision: {np.std(results[2])}\n")
print(f"Average best Recall: {np.mean(results[3])}")
print(f"Standard Deviation Recall: {np.std(results[3])}\n")


BENIGN

In [ ]:
checkpoint_callback_auc1 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/benign_auc1/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_1',
                                                             mode='max',
                                                             save_best_only=True)
checkpoint_callback_auc2 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/benign_auc2/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_2',
                                                             mode='max',
                                                             save_best_only=True)
checkpoint_callback_auc3 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/benign_auc3/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_3',
                                                             mode='max',
                                                             save_best_only=True)
checkpoint_callback_auc4 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/benign_auc4/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_4',
                                                             mode='max',
                                                             save_best_only=True)
checkpoint_callback_auc5 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/benign_auc5/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_5',
                                                             mode='max',
                                                             save_best_only=True)

callbacks_list = [checkpoint_callback_auc1,
                  checkpoint_callback_auc2,
                  checkpoint_callback_auc3,
                  checkpoint_callback_auc4,
                  checkpoint_callback_auc5]

def lr_scheduler(epoch, lr):
    return lr*0.9

lr_decay_function = keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=0)

In [ ]:
histories = []
n_splits = 5
skf = model_selection.StratifiedShuffleSplit(n_splits=n_splits, random_state=314, train_size=0.85)
for number_of_split, data in enumerate(skf.split(X_train, y_train)):
    print(f'SPLIT {number_of_split+1}/{n_splits}:')
    train = [path + dirs[i] for i in data[0]]
    val = [path + dirs[j] for j in data[1]]

    # Generators
    training_generator = DataGenerator(objective=[0, 0, 1, 0],
                                       list_IDs=train,
                                       labels_dir='/kaggle/input/labelsssssss/labels.csv',
                                       dim_img=(192, 256),
                                       batch_size=5,
                                       sub_batch_size=400,
                                       shuffle=True,
                                       training=True)

    validation_generator = DataGenerator(objective=[0, 0, 1, 0],
                                         list_IDs=val,
                                         labels_dir='/kaggle/input/labelsssssss/labels.csv',
                                         dim_img=(192, 256),
                                         batch_size=1,
                                         sub_batch_size='IGNORED', #this argument will be ignored because training is false.
                                         shuffle=True,
                                         training=False)

    densenet = DenseNET()

    densenet.compile(loss='binary_crossentropy',
                     optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                     metrics=[keras.metrics.AUC(name=f'AUC_{number_of_split+1}'),
                              keras.metrics.BinaryAccuracy(name='accuracy'),
                              keras.metrics.Precision(name='precision'),
                              keras.metrics.Recall(name='recall')])


    # Train model on dataset
    histories.append(densenet.fit(training_generator,
                                   validation_data=validation_generator,
                                   epochs=35,
                                   use_multiprocessing=True,
                                   workers=1,
                                   callbacks=[callbacks_list[number_of_split], lr_decay_function]))
    print('\n')
    collect()

In [ ]:
plt.figure(figsize=(25, 50))
plt.subplot(4, 2, 1)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['loss'], marker='o')
    plt.title('DenseNET Loss evolution - Training: Benign vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Loss on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 2)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['val_loss'], marker='o')
    plt.title('DenseNET Loss evolution - Validations: Benign vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Loss on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 3)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['accuracy'], marker='o')
    plt.title('DenseNET Accuracy evolution - Training: Benign vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Accuracy on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 4)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['val_accuracy'], marker='o')
    plt.title('DenseNET Accuracy evolution - Validations: Benign vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Accuracy on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 5)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['recall'], marker='o')
    plt.title('DenseNET Recall evolution - Training: Benign vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Recall on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 6)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['val_recall'], marker='o')
    plt.title('DenseNET Recall evolution - Validations: Benign vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Recall on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 7)
for i, h in enumerate(histories):
    key_auc = f"AUC_{i+1}"
    plt.plot(list(range(1, 36)), h.history[key_auc], marker='o')
    plt.title('DenseNET AUC evolution - Training: Benign vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('AUC on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 8)
for i, h in enumerate(histories):
    key_auc_val = f"val_AUC_{i+1}"
    plt.plot(list(range(1, 36)), h.history[key_auc_val], marker='o')
    plt.title('DenseNET AUC evolution - Validations: Benign vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('AUC on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

In [ ]:
print('Best results in validations for any k-fold: ')
for i, h in enumerate(histories):
    print(f'K-FOLD {i+1}:')
    print("TRAINING RESULTS:")
    k = np.max(h.history[f'AUC_{i+1}'])
    print(f'Best AUC in train: {k}')
    k = np.max(h.history[f'accuracy'])
    print(f'Best Accuracy in train: {k}')
    k = np.max(h.history[f'precision'])
    print(f'Best Precision in train: {k}')
    k = np.max(h.history[f'recall'])
    print(f'Best Recall in train: {k}')

    print("\nVALIDATION RESULTS:")
    k = np.max(h.history[f'val_AUC_{i+1}'])
    print(f'Best AUC in validation: {k}')
    k = np.max(h.history[f'val_accuracy'])
    print(f'Best Accuracy in validation: {k}')
    k = np.max(h.history[f'val_precision'])
    print(f'Best Precision in validation: {k}')
    k = np.max(h.history[f'val_recall'])
    print(f'Best Recall in validation: {k}')
    print()
    print(f'{50*"="}')
    print()

results = np.empty((4, 5))
for i, h in enumerate(histories):
    results[0][i] = np.max(h.history[f'val_AUC_{i+1}'])
    results[1][i] = np.max(h.history['val_accuracy'])
    results[2][i] = np.max(h.history['val_precision'])
    results[3][i] = np.max(h.history['val_recall'])

print(f"Average best AUC: {np.mean(results[0])}")
print(f"standard deviation AUC: {np.std(results[0])}\n")
print(f"Average best Accuracy: {np.mean(results[1])}")
print(f"Standard Deviation Accuracy: {np.std(results[1])}\n")
print(f"Average best Precision: {np.mean(results[2])}")
print(f"Standard Deviation Precision: {np.std(results[2])}\n")
print(f"Average best Recall: {np.mean(results[3])}")
print(f"Standard Deviation Recall: {np.std(results[3])}\n")

CANCER

In [ ]:
checkpoint_callback_auc1 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/cancer_auc1/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_1',
                                                             mode='max',
                                                             save_best_only=True)
checkpoint_callback_auc2 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/cancer_auc2/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_2',
                                                             mode='max',
                                                             save_best_only=True)
checkpoint_callback_auc3 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/cancer_auc3/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_3',
                                                             mode='max',
                                                             save_best_only=True)
checkpoint_callback_auc4 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/cancer_auc4/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_4',
                                                             mode='max',
                                                             save_best_only=True)
checkpoint_callback_auc5 = tf.keras.callbacks.ModelCheckpoint(filepath='./models/cancer_auc5/',
                                                             save_weights_only=True,
                                                             monitor='val_AUC_5',
                                                             mode='max',
                                                             save_best_only=True)

callbacks_list = [checkpoint_callback_auc1,
                  checkpoint_callback_auc2,
                  checkpoint_callback_auc3,
                  checkpoint_callback_auc4,
                  checkpoint_callback_auc5]

def lr_scheduler(epoch, lr):
    return lr*0.9

lr_decay_function = keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=0)

In [ ]:
histories = []
n_splits = 5
skf = model_selection.StratifiedShuffleSplit(n_splits=n_splits, random_state=314, train_size=0.85)
for number_of_split, data in enumerate(skf.split(X_train, y_train)):
    print(f'SPLIT {number_of_split+1}/{n_splits}:')
    train = [path + dirs[i] for i in data[0]]
    val = [path + dirs[j] for j in data[1]]

    # Generators
    training_generator = DataGenerator(objective=[0, 0, 0, 1],
                                       list_IDs=train,
                                       labels_dir='/kaggle/input/labelsssssss/labels.csv',
                                       dim_img=(192, 256),
                                       batch_size=5,
                                       sub_batch_size=400,
                                       shuffle=True,
                                       training=True)

    validation_generator = DataGenerator(objective=[0, 0, 0, 1],
                                         list_IDs=val,
                                         labels_dir='/kaggle/input/labelsssssss/labels.csv',
                                         dim_img=(192, 256),
                                         batch_size=1,
                                         sub_batch_size='IGNORED', #this argument will be ignored because training is false.
                                         shuffle=True,
                                         training=False)

    densenet = DenseNET()

    densenet.compile(loss='binary_crossentropy',
                     optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                     metrics=[keras.metrics.AUC(name=f'AUC_{number_of_split+1}'),
                              keras.metrics.BinaryAccuracy(name='accuracy'),
                              keras.metrics.Precision(name='precision'),
                              keras.metrics.Recall(name='recall')])


    # Train model on dataset
    histories.append(densenet.fit(training_generator,
                                   validation_data=validation_generator,
                                   epochs=35,
                                   use_multiprocessing=True,
                                   workers=1,
                                   callbacks=[callbacks_list[number_of_split], lr_decay_function]))
    print('\n')
    collect()

In [ ]:
plt.figure(figsize=(25, 50))
plt.subplot(4, 2, 1)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['loss'], marker='o')
    plt.title('DenseNET Loss evolution - Training: Cancer vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Loss on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 2)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['val_loss'], marker='o')
    plt.title('DenseNET Loss evolution - Validations: Cancer vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Loss on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 3)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['accuracy'], marker='o')
    plt.title('DenseNET Accuracy evolution - Training: Cancer vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Accuracy on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 4)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['val_accuracy'], marker='o')
    plt.title('DenseNET Accuracy evolution - Validations: Cancer vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Accuracy on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 5)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['recall'], marker='o')
    plt.title('DenseNET Recall evolution - Training: Cancer vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Recall on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 6)
for i, h in enumerate(histories):
    plt.plot(list(range(1, 36)), h.history['val_recall'], marker='o')
    plt.title('DenseNET Recall evolution - Validations: Cancer vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('Recall on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 7)
for i, h in enumerate(histories):
    key_auc = f"AUC_{i+1}"
    plt.plot(list(range(1, 36)), h.history[key_auc], marker='o')
    plt.title('DenseNET AUC evolution - Training: Cancer vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('AUC on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

plt.subplot(4, 2, 8)
for i, h in enumerate(histories):
    key_auc_val = f"val_AUC_{i+1}"
    plt.plot(list(range(1, 36)), h.history[key_auc_val], marker='o')
    plt.title('DenseNET AUC evolution - Validations: Cancer vs others classes')
    plt.xlabel('epoch number')
    plt.ylabel('AUC on epoch')
    plt.legend(['fold-1','fold-2', 'fold-3', 'fold-4', 'fold-5'])

In [ ]:
print('Best results in validations for any k-fold: ')
for i, h in enumerate(histories):
    print(f'K-FOLD {i+1}:')
    print("TRAINING RESULTS:")
    k = np.max(h.history[f'AUC_{i+1}'])
    print(f'Best AUC in train: {k}')
    k = np.max(h.history[f'accuracy'])
    print(f'Best Accuracy in train: {k}')
    k = np.max(h.history[f'precision'])
    print(f'Best Precision in train: {k}')
    k = np.max(h.history[f'recall'])
    print(f'Best Recall in train: {k}')

    print("\nVALIDATION RESULTS:")
    k = np.max(h.history[f'val_AUC_{i+1}'])
    print(f'Best AUC in validation: {k}')
    k = np.max(h.history[f'val_accuracy'])
    print(f'Best Accuracy in validation: {k}')
    k = np.max(h.history[f'val_precision'])
    print(f'Best Precision in validation: {k}')
    k = np.max(h.history[f'val_recall'])
    print(f'Best Recall in validation: {k}')
    print()
    print(f'{50*"="}')
    print()

results = np.empty((4, 5))
for i, h in enumerate(histories):
    results[0][i] = np.max(h.history[f'val_AUC_{i+1}'])
    results[1][i] = np.max(h.history['val_accuracy'])
    results[2][i] = np.max(h.history['val_precision'])
    results[3][i] = np.max(h.history['val_recall'])

print(f"Average best AUC: {np.mean(results[0])}")
print(f"standard deviation AUC: {np.std(results[0])}\n")
print(f"Average best Accuracy: {np.mean(results[1])}")
print(f"Standard Deviation Accuracy: {np.std(results[1])}\n")
print(f"Average best Precision: {np.mean(results[2])}")
print(f"Standard Deviation Precision: {np.std(results[2])}\n")
print(f"Average best Recall: {np.mean(results[3])}")
print(f"Standard Deviation Recall: {np.std(results[3])}\n")

TESTS

NORMAL VS OTHERS


In [ ]:
training_generator = DataGenerator(objective=[1, 0, 0, 0],
                                       list_IDs=X_train,
                                       labels_dir='/kaggle/input/labelsssssss/labels.csv',
                                       dim_img=(192, 256),
                                       batch_size=5,
                                       sub_batch_size=400,
                                       shuffle=True,
                                       training=True)

densenet = DenseNET()

densenet.compile(loss='binary_crossentropy',
                  optimizer=keras.optimizers.Adam(1e-7),
                  metrics=[keras.metrics.AUC(name='AUC'),
                           keras.metrics.BinaryAccuracy(name='accuracy'),
                           keras.metrics.Precision(name='precision'),
                           keras.metrics.Recall(name='recall')])


# Train model on dataset
densenet.load_weights('/kaggle/working/models/normal_auc1/') #start with best AUC in validations
history = densenet.fit(training_generator, epochs=5)

In [ ]:
res = []
for x in X_test:
    x = np.load(x)
    x = np.array(separate_slices(x))
    pred = densenet.predict(np.array(x), verbose=0)
    if (pred.mean() > 0.5):
        res.append(1)
    else:
        res.append(0)
print(res)

In [ ]:
matrix_confusion = np.array([[0, 0], [0, 0]])

for y_pred, y_true in zip(res, y_test):
    if y_true != 1:
        y_true = 0
    else:
        y_true = 1

    matrix_confusion[y_pred][y_true] += 1
print(matrix_confusion)
print((matrix_confusion[0][0] + matrix_confusion[1][1])/matrix_confusion.sum())

Actionable vs others

In [ ]:
training_generator = DataGenerator(objective=[0, 1, 0, 0],
                                       list_IDs=X_train,
                                       labels_dir='/kaggle/input/labelsssssss/labels.csv',
                                       dim_img=(192, 256),
                                       batch_size=5,
                                       sub_batch_size=400,
                                       shuffle=True,
                                       training=True)

densenet = DenseNET()

densenet.compile(loss='binary_crossentropy',
                  optimizer=keras.optimizers.Adam(1e-7),
                  metrics=[keras.metrics.AUC(name='AUC'),
                           keras.metrics.BinaryAccuracy(name='accuracy'),
                           keras.metrics.Precision(name='precision'),
                           keras.metrics.Recall(name='recall')])


# Train model on dataset
densenet.load_weights('/kaggle/working/models/actionable_auc1/') #start with best AUC in validations
history = densenet.fit(training_generator, epochs=5)

In [ ]:
res = []
for x in X_test:
    x = np.load(x)
    x = np.array(separate_slices(x))
    pred = densenet.predict(np.array(x), verbose=0)
    if (pred.mean() > 0.5):
        res.append(1)
    else:
        res.append(0)
print(res)

In [ ]:
matrix_confusion = np.array([[0, 0], [0, 0]])

for y_pred, y_true in zip(res, y_test):
    if y_true != 1:
        y_true = 0
    else:
        y_true = 1

    matrix_confusion[y_pred][y_true] += 1
print(matrix_confusion)
print((matrix_confusion[0][0] + matrix_confusion[1][1])/matrix_confusion.sum())

BENIGN VS OTHERS

In [ ]:
training_generator = DataGenerator(objective=[0, 0, 1, 0],
                                       list_IDs=X_train,
                                       labels_dir='/kaggle/input/labelsssssss/labels.csv',
                                       dim_img=(192, 256),
                                       batch_size=5,
                                       sub_batch_size=400,
                                       shuffle=True,
                                       training=True)

densenet = DenseNET()

densenet.compile(loss='binary_crossentropy',
                  optimizer=keras.optimizers.Adam(1e-7),
                  metrics=[keras.metrics.AUC(name='AUC'),
                           keras.metrics.BinaryAccuracy(name='accuracy'),
                           keras.metrics.Precision(name='precision'),
                           keras.metrics.Recall(name='recall')])


# Train model on dataset
densenet.load_weights('/kaggle/working/models/benign_auc5/') #start with best AUC in validations
history = densenet.fit(training_generator, epochs=5)

In [ ]:
res = []
for x in X_test:
    x = np.load(x)
    x = np.array(separate_slices(x))
    pred = densenet.predict(np.array(x), verbose=0)
    if (pred.mean() > 0.5):
        res.append(1)
    else:
        res.append(0)
print(res)

In [ ]:
matrix_confusion = np.array([[0, 0], [0, 0]])

for y_pred, y_true in zip(res, y_test):
    if y_true != 2:
        y_true = 0
    else:
        y_true = 1

    matrix_confusion[y_pred][y_true] += 1
print(matrix_confusion)
print((matrix_confusion[0][0] + matrix_confusion[1][1])/matrix_confusion.sum())

CANCER VS OTHERS


In [ ]:
training_generator = DataGenerator(objective=[0, 0, 0, 1],
                                       list_IDs=X_train,
                                       labels_dir='/kaggle/input/labelsssssss/labels.csv',
                                       dim_img=(192, 256),
                                       batch_size=5,
                                       sub_batch_size=400,
                                       shuffle=True,
                                       training=True)

densenet = DenseNET()

densenet.compile(loss='binary_crossentropy',
                  optimizer=keras.optimizers.Adam(1e-7),
                  metrics=[keras.metrics.AUC(name='AUC'),
                           keras.metrics.BinaryAccuracy(name='accuracy'),
                           keras.metrics.Precision(name='precision'),
                           keras.metrics.Recall(name='recall')])


# Train model on dataset
densenet.load_weights('/kaggle/working/models/cancer_auc3/') #start with best AUC in validations
history = densenet.fit(training_generator, epochs=5)

In [ ]:
res = []
for x in X_test:
    x = np.load(x)
    x = np.array(separate_slices(x))
    pred = densenet.predict(np.array(x), verbose=0)
    if (pred.mean() > 0.5):
        res.append(1)
    else:
        res.append(0)
print(res)

In [ ]:
matrix_confusion = np.array([[0, 0], [0, 0]])

for y_pred, y_true in zip(res, y_test):
    if y_true != 3:
        y_true = 0
    else:
        y_true = 1

    matrix_confusion[y_pred][y_true] += 1
print(matrix_confusion)
print((matrix_confusion[0][0] + matrix_confusion[1][1])/matrix_confusion.sum())